In [ ]:
# Install required packages
!pip install tensorflow_datasets
!pip install tensorflow_addons
!pip install matplotlib

In [ ]:
!pip install git+https://github.com/tensorflow/examples.git

In [5]:
# Import required libraries
import tensorflow as tf
#from tensorflow_examples.models.deeplab import DeepLabV3Plus
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Load the Cityscapes dataset
dataset, info = tfds.load('cityscapes/semantic_segmentation', split='train', with_info=True)
num_classes = info.features['label'].num_classes

In [ ]:
# Define preprocessing function for the images and labels
def preprocess_image(image, label):
    image = tf.image.resize(image, (512, 512))
    image = tf.cast(image, tf.float32)
    image /= 255.0
    label -= 1  # Shift the label values to start from 0
    return image, label

In [ ]:
# Apply the preprocessing function to the dataset
dataset = dataset.map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
dataset = dataset.batch(8)
dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)

# Define the DeepLabV3+ model
deeplab_model = DeepLabV3Plus(input_shape=(512, 512, 3), classes=num_classes)

# Compile the model
deeplab_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
                      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                      metrics=['accuracy'])

In [ ]:
# Train the model
history = deeplab_model.fit(dataset, epochs=20)

# Plot the training accuracy and loss
acc = history.history['accuracy']
loss = history.history['loss']
epochs_range = range(20)
plt.figure(figsize=(15, 5))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.legend(loc='lower right')
plt.title('Training Accuracy')
plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.legend(loc='upper right')
plt.title('Training Loss')
plt.show()